In [1]:
from utils.spark_session import get_spark_session
from utils.hadoop_setup import complete_hadoop_setup

# Hadoop complete setup
complete_hadoop_setup()

# Create Spark Session and assign to spark 'variable'
spark = get_spark_session()

✔ HADOOP_HOME set to:, os.environ['HADOOP_HOME']
✔ Added to PATH: C:\hadoop\bin

 ✔ winutils.exe: True
 ✔ hadoop.dll: True

🎉 Setup complete!
✔ HADOOP_HOME set to:, os.environ['HADOOP_HOME']
✔ Added to PATH: C:\hadoop\bin

 ✔ winutils.exe: True
 ✔ hadoop.dll: True

🎉 Setup complete!


In [23]:
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\chira\Desktop\data_engineering\PySpark\nyc-taxi-analytics-platform")

SILVER_GREEN_PATH = str(PROJECT_ROOT / "data" / "silver" / "nyc_taxi" / "green")
GOLD_BASE_PATH = PROJECT_ROOT / "data" / "gold" / "nyc_taxi" / "green"

GOLD_DAILY_PATH = str(GOLD_BASE_PATH / "daily_metrics")
GOLD_HOURLY_PATH = str(GOLD_BASE_PATH / "hourly_metrics")

In [24]:
# Read Silver

df_silver = spark.read.format("delta").load(SILVER_GREEN_PATH)

print(f"Silver row count: {df_silver.count():,}")

Silver row count: 47,364


## Gold Daily Metrics (PySpark)

In [25]:
# Daily Aggregation (DataFrame API)

from pyspark.sql.functions import (
    to_date,
    count,
    sum as spark_sum,
    avg
)

df_daily = (
    df_silver
    .withColumn("trip_date", to_date("pickup_ts"))
    .groupBy("trip_date", "year", "month")
    .agg(
        count("*").alias("total_trips"),
        spark_sum("total_amount").alias("total_revenue"),
        spark_sum("fare_amount").alias("total_fare"),
        avg("trip_distance").alias("avg_trip_distance"),
        avg("fare_amount").alias("avg_fare_amount"),
        avg("total_amount").alias("avg_total_amount")
    )
)

In [26]:
(
    df_daily
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("year", "month")
    .save(GOLD_DAILY_PATH)
)

print("✔ Gold daily metrics written")

✔ Gold daily metrics written


In [27]:
df_daily.show()

+----------+----+-----+-----------+------------------+------------------+------------------+------------------+------------------+
| trip_date|year|month|total_trips|     total_revenue|        total_fare| avg_trip_distance|   avg_fare_amount|  avg_total_amount|
+----------+----+-----+-----------+------------------+------------------+------------------+------------------+------------------+
|2025-09-01|2025|    9|       1312| 41135.46999999999|31339.950000000037| 11.18649390243903| 23.88715701219515|31.353254573170723|
|2025-09-02|2025|    9|       1654| 49120.58999999996|35456.520000000135| 9.380284159613058|21.436831922611933|29.698059250302272|
|2025-09-19|2025|    9|       1716|46974.559999999896| 32368.00000000014|  93.2456934731935|18.862470862470943|27.374452214452155|
|2025-09-29|2025|    9|       1647| 41678.23999999992|27124.250000000207|3.4698360655737726|  16.4688828172436|25.305549483910088|
|2025-09-03|2025|    9|       1745| 57483.45000000003|  42060.1700000001| 4.1664183

## Gold Hourly Metrics (PySpark)

In [28]:
# Hour Extractions

from pyspark.sql.functions import hour

df_hourly = (
    df_silver
    .withColumn("trip_date", to_date("pickup_ts"))
    .withColumn("trip_hour", hour("pickup_ts"))
    .groupBy("trip_date", "trip_hour", "year", "month")
    .agg(
        count("*").alias("trips_per_hour"),
        spark_sum("total_amount").alias("revenue_per_hour"),
        avg("fare_amount").alias("avg_fare_per_hour")
    )
)

In [29]:
(
    df_hourly
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("year", "month")
    .save(GOLD_HOURLY_PATH)
)

print("✔Gold hourly metrics written")

✔Gold hourly metrics written


In [30]:
df_hourly.show()

+----------+---------+----+-----+--------------+------------------+------------------+
| trip_date|trip_hour|year|month|trips_per_hour|  revenue_per_hour| avg_fare_per_hour|
+----------+---------+----+-----+--------------+------------------+------------------+
|2025-09-01|        2|2025|    9|            14|            735.41| 42.91428571428572|
|2025-09-01|        7|2025|    9|            15|            388.59|19.255333333333333|
|2025-09-01|        9|2025|    9|            26|            573.76|16.709615384615383|
|2025-09-01|       10|2025|    9|            42|            1013.2|17.623333333333328|
|2025-09-02|       14|2025|    9|            90|2184.7899999999995| 17.50411111111112|
|2025-09-02|       15|2025|    9|           103|3288.6000000000013|23.967766990291263|
|2025-09-02|       17|2025|    9|           140| 3990.419999999999| 19.55178571428572|
|2025-09-02|       19|2025|    9|            92|           2691.34|19.971521739130434|
|2025-09-02|        4|2025|    9|          

## Same Logic Using Spark SQL

In [31]:
# Create a Temporary View

df_silver.createOrReplaceTempView("silver_green")

In [32]:
# Daily Metrics (SQL)

df_daily_sql = spark.sql("""
SELECT
    TO_DATE(pickup_ts) AS trip_date,
    year,
    month,
    COUNT(*) AS total_trips,
    SUM(total_amount) AS total_revenue,
    SUM(fare_amount) AS total_fare,
    AVG(trip_distance) AS avg_trip_distance,
    AVG(fare_amount) AS avg_fare_amount,
    AVG(total_amount) AS avg_total_amount
FROM silver_green
GROUP BY TO_DATE(pickup_ts), year, month
""")

In [33]:
# Write

(
    df_daily_sql
    .write
    .mode("overwrite")
    .partitionBy("year", "month")
    .save(str(GOLD_BASE_PATH / "daily_metrics_sql"))
)

In [34]:
df_daily_sql.show()

+----------+----+-----+-----------+------------------+------------------+------------------+------------------+------------------+
| trip_date|year|month|total_trips|     total_revenue|        total_fare| avg_trip_distance|   avg_fare_amount|  avg_total_amount|
+----------+----+-----+-----------+------------------+------------------+------------------+------------------+------------------+
|2025-09-01|2025|    9|       1312| 41135.46999999999|31339.950000000037| 11.18649390243903| 23.88715701219515|31.353254573170723|
|2025-09-02|2025|    9|       1654| 49120.58999999996|35456.520000000135| 9.380284159613058|21.436831922611933|29.698059250302272|
|2025-09-19|2025|    9|       1716|46974.559999999896| 32368.00000000014|  93.2456934731935|18.862470862470943|27.374452214452155|
|2025-09-29|2025|    9|       1647| 41678.23999999992|27124.250000000207|3.4698360655737726|  16.4688828172436|25.305549483910088|
|2025-09-03|2025|    9|       1745| 57483.45000000003|  42060.1700000001| 4.1664183

In [36]:
# Hourly Metrics (SQL)

df_hourly_sql = spark.sql("""
SELECT
    TO_DATE(pickup_ts) AS trip_date,
    HOUR(pickup_ts) AS trip_hour,
    year,
    month,
    COUNT(*) AS total_trips,
    SUM(total_amount) AS revenue_per_hour,
    AVG(fare_amount) AS avg_fare_amount
FROM silver_green
GROUP BY TO_DATE(pickup_ts), HOUR(pickup_ts), month, year
""")

In [38]:
# Write

(
    df_hourly_sql
    .write
    .mode("overwrite")
    .partitionBy("year", "month")
    .save(str(GOLD_BASE_PATH / "hourly_metrics_sql"))
)

In [39]:
df_hourly_sql.show()

+----------+---------+----+-----+-----------+------------------+------------------+
| trip_date|trip_hour|year|month|total_trips|  revenue_per_hour|   avg_fare_amount|
+----------+---------+----+-----+-----------+------------------+------------------+
|2025-09-01|        3|2025|    9|         11|493.05999999999995|34.527272727272724|
|2025-09-02|        7|2025|    9|         77|           1712.21|15.583766233766246|
|2025-09-01|        8|2025|    9|         11|268.71000000000004|16.566363636363636|
|2025-09-01|       11|2025|    9|         53|1131.2200000000003|15.616981132075468|
|2025-09-02|       14|2025|    9|         90|2184.7899999999995| 17.50411111111112|
|2025-09-01|       16|2025|    9|        127|           4266.18| 25.55062992125984|
|2025-09-02|       17|2025|    9|        140| 3990.419999999999| 19.55178571428572|
|2025-09-01|       19|2025|    9|         86|           2702.89|23.377441860465122|
|2025-09-02|       18|2025|    9|        136| 4097.070000000001|21.691544117